## 3. Estimating and identifying risk

In [1]:
# Importing required modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader.data as web
from scipy.stats import norm, anderson

In [2]:
#upload
tickers=['GS','C','JPM','BAC']
start_date='2005-01-01'
end_date='2010-12-31'

In [3]:
df = web.DataReader(tickers,'yahoo',start=start_date,end=end_date)
stocks=df['Close']

In [4]:
stock_returns=stocks.pct_change().dropna()
weight=[0.25,0.25,0.25,0.25]

In [5]:
portfolio_losses=stock_returns@weight

Parameter estimation: Normal <br>
Is a Normal distribution a good fit? You'll test this with the scipy.stats.anderson Anderson-Darling test. If the test result is statistically different from zero, this indicates the data is not Normally distributed. You'll address this in the next exercise.

In [7]:
# Fit portfolio losses to the Normal distribution
params = norm.fit(portfolio_losses)

# Compute the 95% VaR from the fitted distribution, using parameter estimates
VaR_95 = norm.ppf(0.95, *params)
print("VaR_95, Normal distribution: ", VaR_95)

# Test the data for Normality
print("Anderson-Darling test result: ", anderson(portfolio_losses))

VaR_95, Normal distribution:  0.055567342382951385
Anderson-Darling test result:  AndersonResult(statistic=92.77546604613713, critical_values=array([0.574, 0.654, 0.785, 0.916, 1.089]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]))


Parameter estimation: Skewed Normal <br>
Now you'll parametrically estimate the 95% VaR of a loss distribution fit using scipy.stats's skewnorm skewed Normal distribution. This is a more general distribution than the Normal and allows losses to be non-symmetrically distributed. We might expect losses to be skewed during the crisis, when portfolio losses were more likely than gains.

In [8]:
# Import the skew-normal distribution and skewness test from scipy.stats
from scipy.stats import skewnorm, skewtest

# Test the data for skewness
print("Skewtest result: ", skewtest(portfolio_losses))

# Fit the portfolio loss data to the skew-normal distribution
params = skewnorm.fit(portfolio_losses)

# Compute the 95% VaR from the fitted distribution, using parameter estimates
VaR_95 = skewnorm.ppf(0.95, *params)
print("VaR_95 from skew-normal: ", VaR_95)

Skewtest result:  SkewtestResult(statistic=16.107584500407846, pvalue=2.2566846813270645e-58)
VaR_95 from skew-normal:  0.05720213517341201


Historical Simulation <br>

In [10]:
# Find the historical simulated VaR estimates
VaR_95 = [np.quantile(x, 0.95) for x in portfolio_losses]

# Display the VaR estimates
print("VaR_95, 2005-2006: ", VaR_95[0], '; VaR_95, 2007-2009: ', VaR_95[1])

VaR_95, 2005-2006:  -0.011912540009759998 ; VaR_95, 2007-2009:  -0.0003637596297078327
